In [45]:
# You should start your notebook with:  JULIA_NUM_THREADS=4 jupyter notebook
Base.Threads.nthreads()
using BenchmarkTools

In [2]:
N = 1000

A = rand(N,N)
B = rand(N,N)
C = A *B;

### Naive matrix multiply

- select row `r` in A, let's call it `row_r`

- select col `c` in B, let's call it `col_c`

- compute element `C[r,c]` as the scalar product of `row_ by` `row_c`



In [3]:
C2 = zeros(size(A));


function dot_product_row_r_col_c(A,r,B,c, n_elements)
    value = zero(eltype(A))
    for i in 1:n_elements
        value += A[r,i] * B[i,c] 
    end
    return value    
end 


dot_product_row_r_col_c (generic function with 1 method)

In [4]:

"""
Naive matrix multiply of A and B
"""
function naive_matmul!(A, B, C)
    n_rows_A = size(A,1)
    n_cols_B = size(B,2)
    n_elements = n_cols_B
        
    for r in 1:n_rows_A
        for c in 1:n_cols_B
           C[r,c] = dot_product_row_r_col_c(A,r,B,c, n_elements)
        end
    end
    
end

naive_matmul!

In [46]:
@btime naive_matmul!(A,B,C2)

  880.758 ms (0 allocations: 0 bytes)


In [47]:
@btime C = A * B;

  7.090 ms (2 allocations: 7.63 MiB)


In [7]:
isapprox(C,C2;rtol=0.000001)

true

### Accessing columns in the transposed of `A` instead of rows in `A`

Notice that the elements of an Array in Julia are stored in memory by column. 

This makes the memory access of `A[r,i]` in the function `dot_product_row_r_col_c` slow (where `i` is changing).

We can improve this by by accessing `A[i,r]` in a new function `dot_product_col_r_col_c`. To have an equivalent computation we simply transpose `A` and then select a row of `A` (a column of `A_t`) and do the dot product with a column of `B`.

In [8]:
C2 = zeros(size(A));

function dot_product_col_r_col_c(A,r,B,c, n_elements)
    value = zero(eltype(A))
    
    for i in 1:n_elements
        value += A[i,r] * B[i,c] 
    end
    return value    
end 

dot_product_col_r_col_c (generic function with 1 method)

In [9]:

"""
Naive matrix multiply of A and B
"""
function naive_matmul_2!(A, B, C)
    n_rows_A = size(A,1)
    n_cols_B = size(B,2)
    n_elements = n_cols_B
    A_t = copy(transpose(A))
    for r in 1:n_rows_A
        for c in 1:n_cols_B
           C[r,c] = dot_product_col_r_col_c(A_t,r,B,c, n_elements)
        end
    end
    
end

naive_matmul_2!

In [48]:
@btime naive_matmul_2!(A,B,C2)

  900.415 ms (2 allocations: 7.63 MiB)


In [11]:
isapprox(C,C2;rtol=0.000001)

true

Notice that even taking into account that we make a copy of A inside the function this version is twice as fast.


#### Using SIMD instructions

The function `dot_product_col_r_col_c` can benefit from SIMD instructions

In [12]:


@inline function dot_product_col_r_col_c_simd(A,r,B,c, n_elements)
    value = zero(eltype(A))
    
    @simd for i in 1:n_elements
       @inbounds value += A[i,r] * B[i,c] 
    end
    return value    
end 



dot_product_col_r_col_c_simd (generic function with 1 method)

In [13]:

function naive_matmul_3!(A, B, C)
    n_rows_A = size(A,1)
    n_cols_B = size(B,2)
    n_elements = n_cols_B
    A_t = copy(transpose(A))
    for r in 1:n_rows_A
        for c in 1:n_cols_B
           C[r,c] = dot_product_col_r_col_c_simd(A_t,r,B,c, n_elements)
        end
    end
end

naive_matmul_3! (generic function with 1 method)

In [49]:
C2 = zeros(size(A));

@btime naive_matmul_3!(A,B,C2)

  251.343 ms (2 allocations: 7.63 MiB)


In [50]:
isapprox(C,C2;rtol=0.000001)

true

#### Blocked version version

In [52]:
C2 = zeros(size(A));

function block_times_block(A, r, B, c, block_size)
    value = zero(eltype(A))
    
    for block_r in r:r+block_size
        for block_c in c:c+block_size
            @inbounds value += A[r,c] * B[r,c]  
        end
    end
    return value    
end 



block_times_block (generic function with 1 method)

In [53]:

function naive_matmul_4!(A, B, C)
    n_rows_A = size(A,1)
    n_cols_B = size(B,2)
    n_elements = n_cols_B
    bs = 10
    n_elements_block = bs*bs
    
    for row in 1:bs:n_rows_A-bs
        for col in 1:bs:n_cols_B-bs
            for block_row in row:(row+bs)
                for block_col in col:(col+bs)
                    res = 0.0
                    #C[row:row+bs,col:col+bs] = block_times_block(A, row, B, col, n_elements_block)
                     @inbounds @fastmath for k in 1:n_elements_block
                        res += A[k, block_row] * B[block_col, k]
                    end
                    C[row, col] = res
                end
            end
        end
    end
    
end

naive_matmul_4! (generic function with 1 method)

In [54]:
C2 = zeros(size(A));

@btime naive_matmul_4!(A,B,C2)

  81.257 ms (0 allocations: 0 bytes)


In [59]:
isapprox(C,C2;rtol=0.000001)

false

### Multithreaded version 

In [60]:

function naive_matmul_5!(A, B, C)
    n_rows_A = size(A,1)
    n_cols_B = size(B,2)
    n_elements = n_cols_B
    bs = 10
    n_elements_block = bs*bs
    
    for row in 1:bs:n_rows_A-bs
        Threads.@threads for col in 1:bs:n_cols_B-bs
            for block_row in row:(row+bs)
                @inbounds @fastmath for block_col in col:(col+bs)
                    res = 0.0
                    #C[row:row+bs,col:col+bs] = block_times_block(A, row, B, col, n_elements_block)
                    for k in 1:n_elements_block
                        res += A[k, block_row] * B[block_col, k]
                    end
                    C[row, col] = res
                end
            end
        end
    end
    
end

naive_matmul_5! (generic function with 1 method)

In [61]:
C2 = zeros(size(A));

@btime naive_matmul_5!(A,B,C2)

  19.929 ms (6058 allocations: 610.06 KiB)


Notice that A*B is still 10x faster

In [51]:
@btime A*B;

  7.082 ms (2 allocations: 7.63 MiB)
